In [1]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import h5py
from PIL import Image
from src.benchmarkers import *
from src.benchmarkersV2 import *
from tqdm import tqdm
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from io import BytesIO

np.random.seed(0)

In [29]:
table = pq.read_table(r"C:\Users\Cristiano Lavoro\Downloads\ds_10.parquet")

In [21]:
struct_array = table.column("image_feature").chunk(0).values.field("boundingbox_feature").values

In [24]:
current_schema = struct_array.type
current_schema

StructType(struct<bbox: list<element: double>, image_1_feature: list<element: struct<image: binary, shape: list<element: int64>>>, label_feature: list<element: struct<label: int64>>>)

In [25]:
new_data = [
    struct_array.field(f.name) for f in current_schema
] + [pa.array([40 for item in range(50)])]

In [28]:
new_struct_array = pa.StructArray.from_arrays(new_data, fields=reconstructed_schema)

ArrowInvalid: Mismatching number of fields and child arrays

In [30]:
import pyarrow as pa
import json

# Define the schema with a list of struct fields
#schema = pa.schema([
#    pa.field('id', pa.int32()),
#    pa.field('name', pa.string()),
#    pa.field('addresses', pa.list_(
#        pa.struct([
#            ('street', pa.list_(pa.struct([("prova",pa.string())]))),
#            ('city', pa.string()),
#            ('zipcode', pa.int32())
#        ])
#    ))
#])
schema = table.schema

# Convert pyarrow schema to a JSON serializable format
def schema_to_dict(schema):
    schema_dict = {}
    for field in schema:
        schema_dict[field.name] = field_to_dict(field)
    return schema_dict

def field_to_dict(field):
    if pa.types.is_list(field.type):
        # For list types, extract the value field type (should be a struct in your case)
        return {
            "type": "list",
            "value_type": field_to_dict(field.type.value_field)
        }
    elif pa.types.is_struct(field.type):
        # For struct types, extract the fields
        return {
            "type": "struct",
            "fields": {f.name: field_to_dict(f) for f in field.type}
        }
    else:
        # For primitive types (int, string, etc.)
        return {"type": str(field.type)}

# Convert to JSON
schema_json = json.dumps(schema_to_dict(schema), indent=2)
schema_dict = schema_to_dict(schema)
print(type(schema_dict))
schema_dict

<class 'dict'>


{'image_feature': {'type': 'list',
  'value_type': {'type': 'struct',
   'fields': {'boundingbox_feature': {'type': 'list',
     'value_type': {'type': 'struct',
      'fields': {'bbox': {'type': 'list', 'value_type': {'type': 'double'}},
       'image_1_feature': {'type': 'list',
        'value_type': {'type': 'struct',
         'fields': {'image': {'type': 'binary'},
          'shape': {'type': 'list', 'value_type': {'type': 'int64'}}}}},
       'label_feature': {'type': 'list',
        'value_type': {'type': 'struct',
         'fields': {'label': {'type': 'int64'}}}}}}},
    'image': {'type': 'binary'},
    'shape': {'type': 'list', 'value_type': {'type': 'int64'}},
    'text_1_feature': {'type': 'list',
     'value_type': {'type': 'struct',
      'fields': {'text': {'type': 'string'}}}}}}},
 'text_feature': {'type': 'list',
  'value_type': {'type': 'struct', 'fields': {'text': {'type': 'string'}}}}}

In [37]:
import pyarrow as pa
import json

# A mapping from JSON string types to pyarrow types
TYPE_MAPPING = {
    'int64': pa.int64,
    'int32': pa.int32,
    'float64': pa.float64,  # This is the correct PyArrow type for 'double'
    'double': pa.float64,   # If 'double' is used in the JSON, map it to 'float64'
    'string': pa.string,
    'binary': pa.binary
}

# Convert JSON back to PyArrow Schema
def dict_to_field(name, field_dict):
    field_type = field_dict['type']
    
    if field_type == 'list':
        value_type_dict = field_dict['value_type']
        
        # Check if the list contains a struct or a primitive type
        if 'fields' in value_type_dict:
            # List of struct
            value_type = pa.struct([
                dict_to_field(f_name, f_dict)
                for f_name, f_dict in value_type_dict['fields'].items()
            ])
        else:
            # List of primitive types
            value_type = TYPE_MAPPING[value_type_dict['type']]()

        return pa.field(name, pa.list_(value_type))

    elif field_type == 'struct':
        return pa.field(name, pa.struct([
            dict_to_field(f_name, f_dict)
            for f_name, f_dict in field_dict['fields'].items()
        ]))
    
    else:
        # For primitive types (int, string, binary, etc.)
        return pa.field(name, TYPE_MAPPING[field_type]())

# Convert JSON back to PyArrow Schema
def dict_to_schema(schema_dict):
    fields = [dict_to_field(name, field_dict) for name, field_dict in schema_dict.items()]
    return pa.schema(fields)

# Example: JSON string back to schema
schema_dict = {
  "image_feature": {
    "type": "list",
    "value_type": {
      "type": "struct",
      "fields": {
        "boundingbox_feature": {
          "type": "list",
          "value_type": {
            "type": "struct",
            "fields": {
              "bbox": {
                "type": "list",
                "value_type": {
                  "type": "double"
                }
              },
              "area":{
                "type":"double"  
              },
              "image_1_feature": {
                "type": "list",
                "value_type": {
                  "type": "struct",
                  "fields": {
                    "image": {
                      "type": "binary"
                    },
                    "shape": {
                      "type": "list",
                      "value_type": {
                        "type": "int64"
                      }
                    }
                  }
                }
              },
              "label_feature": {
                "type": "list",
                "value_type": {
                  "type": "struct",
                  "fields": {
                    "label": {
                      "type": "int64"
                    }
                  }
                }
              }
            }
          }
        },
        "image": {
          "type": "binary"
        },
        "shape": {
          "type": "list",
          "value_type": {
            "type": "int64"
          }
        },
        "text_1_feature": {
          "type": "list",
          "value_type": {
            "type": "struct",
            "fields": {
              "text": {
                "type": "string"
              }
            }
          }
        }
      }
    }
  },
  "text_feature": {
    "type": "list",
    "value_type": {
      "type": "struct",
      "fields": {
        "text": {
          "type": "string"
        }
      }
    }
  }
}

# Reconstruct schema
reconstructed_schema = dict_to_schema(schema_dict)
print(reconstructed_schema)


image_feature: list<item: struct<boundingbox_feature: list<item: struct<bbox: list<item: double>, area: double, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int64>>>>, image: binary, shape: list<item: int64>, text_1_feature: list<item: struct<text: string>>>>
  child 0, item: struct<boundingbox_feature: list<item: struct<bbox: list<item: double>, area: double, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int64>>>>, image: binary, shape: list<item: int64>, text_1_feature: list<item: struct<text: string>>>
      child 0, boundingbox_feature: list<item: struct<bbox: list<item: double>, area: double, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int64>>>>
          child 0, item: struct<bbox: list<item: double>, area: double, image_1_feature: list<item: struct<image: binary, 

In [38]:
table = table.cast(reconstructed_schema)

ArrowTypeError: struct fields don't match or are in the wrong order: Input fields: struct<bbox: list<item: double>, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int64>>> output fields: struct<bbox: list<item: double>, area: double, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int64>>>

In [36]:
table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("area")

KeyError: 'area'